(profiling)=
# Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools which are wrapped in PyMC3 by {func}`model.profile <pymc.model.core.Model.profile>`. This function returns a `ProfileStats` object conveying information about the underlying Theano operations. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [1]:
import numpy as np
import pandas as pd
import pymc as pm

print(f"Running on PyMC3 v{pm.__version__}")

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running on PyMC3 v5.10.0


In [2]:
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)

In [3]:
# Load the data
returns = pd.read_csv(pm.get_data("SP500.csv"), index_col=0, parse_dates=True)

In [4]:
# Stochastic volatility example
with pm.Model() as model:
    sigma = pm.Exponential("sigma", 1.0 / 0.02, initval=0.1)
    nu = pm.Exponential("nu", 1.0 / 10)
    s = pm.GaussianRandomWalk("s", sigma**-2, shape=returns.shape[0])
    r = pm.StudentT("r", nu, lam=np.exp(-2 * s), observed=returns["change"])

C:\Users\Divya Tiwari\anaconda3\envs\pymc_env\Lib\site-packages\pymc\distributions\timeseries.py:293: UserWarning: Initial distribution not specified, defaulting to `Normal.dist(0, 100)`.You can specify an init_dist manually to suppress this warning.
  warnings.warn(


Then we call the `profile` function and summarize its return values.

In [5]:
# Profiling of the logp call
model.profile(model.logp()).summary()

Function profiling
  Message: C:\Users\Divya Tiwari\anaconda3\envs\pymc_env\Lib\site-packages\pymc\pytensorf.py:990
  Time in 1000 calls to Function.__call__: 1.331489e-01s
  Time in Function.vm.__call__: 0.10964459989918396s (82.347%)
  Time in thunks: 0.1032404899597168s (77.538%)
  Total compilation time: 9.425617e-01s
    Number of Apply nodes: 26
    PyTensor rewrite time: 8.539493e-01s
       PyTensor validate time: 2.782300e-03s
    PyTensor Linker time (includes C, CUDA code generation/compiling): 0.07904159999452531s
       Import time 3.465450e-02s
       Node make_thunk time 7.485420e-02s
           Node Composite{...}(s, [1.0435322 ... 54666e-07], ExpandDims{axis=0}.0, Composite{...}.0, Composite{...}.1, Composite{gammaln((0.5 * i0))}.0) time 4.614200e-03s
           Node Composite{...}(sigma_log__, 50.00000111758712, 0.0, 3.912023027779888, -inf) time 4.260600e-03s
           Node Composite{reciprocal(sqr(i0))}(ExpandDims{axis=0}.0) time 4.128100e-03s
           Node MakeV

In [6]:
# Profiling of the gradient call dlogp/dx
model.profile(pm.gradient(model.logp(), vars=None)).summary()

Function profiling
  Message: C:\Users\Divya Tiwari\anaconda3\envs\pymc_env\Lib\site-packages\pymc\pytensorf.py:990
  Time in 1000 calls to Function.__call__: 1.622821e-01s
  Time in Function.vm.__call__: 0.13946900120936334s (85.942%)
  Time in thunks: 0.13289427757263184s (81.891%)
  Total compilation time: 9.082990e-01s
    Number of Apply nodes: 34
    PyTensor rewrite time: 8.388561e-01s
       PyTensor validate time: 8.517501e-03s
    PyTensor Linker time (includes C, CUDA code generation/compiling): 0.059757000009994954s
       Import time 2.345270e-02s
       Node make_thunk time 5.872960e-02s
           Node Composite{...}(s, ExpandDims{axis=0}.0, [1.0435322 ... 54666e-07], ExpandDims{axis=0}.0) time 4.175900e-03s
           Node Composite{...}(nu_log__, 1.0, 0.5, -1453.0, 1453.0, -0.1, 0.0, 0) time 4.137400e-03s
           Node Composite{(i4 + i5 + (i0 * i1 * i2 * i3))}(-2.0, Sum{axes=None}.0, Composite{...}.1, Composite{...}.0, Composite{...}.2, 1.0) time 3.595000e-03s
     

In [7]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Mon Dec 04 2023

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.15.0

pandas: 2.1.3
pymc  : 5.10.0
numpy : 1.26.2

Watermark: 2.4.3

